# CCS Calibration

To yield collision cross section (CCS) from ion mobility spectrometry (IMS) drift time, a calibration must be performed using a standard tune mix containing compounds of known CCS. Drift times, or analogous measurement, such as inverse reduced mobility in trapped ion mobility spectrometry (TIMS), are reported by the instrument and calibrated against the known CCS values to yield calibration coefficients `beta` and `tfix`.

## Drift Tube and Trapped IMS

For drift tube and trapped IMS, the single-field calibration equation detailed in Stow et al. 2017 is implemented.
Provided reference *m/z* and CCS, buffer gass mass (typically N2), as well as experimental drift time (or analogous measurement), the calibration is performed as follows.

In [1]:
import deimos

In [2]:
# Perform calibration
ccs_cal_pos = deimos.calibration.calibrate_ccs(mz=[118.086255, 322.048121, 622.028960, 922.009798, 1221.990636, 1521.971475],
                                               ccs=[121.3, 153.7, 203, 243.6, 282.2, 317],
                                               q=[1, 1, 1, 1, 1, 1],
                                               ta=[14.078552, 19.13204, 25.799976, 31.193768, 36.225488, 40.712136],
                                               buffer_mass=28.013)

# R-squared
print('r-squared:\t', ccs_cal_pos.fit['r'] ** 2)

r-squared:	 0.9999784552958134


In [3]:
# Perform calibration
ccs_cal_neg = deimos.calibration.calibrate_ccs(mz=[301.998139, 601.978977, 1033.988109, 1333.968947, 1633.949786],
                                               ccs=[140, 180.8, 255.3, 284.8, 319],
                                               q=[1, 1, 1, 1, 1],
                                               ta=[17.40552, 22.96208, 32.718984, 36.637792, 41.08044],
                                               buffer_mass=28.013)

# R-squared
print('r-squared:\t', ccs_cal_pos.fit['r'] ** 2)

r-squared:	 0.9999784552958134


The above requires detection of relevant ions in the tune mix data, which must be determined through peak detection or manual exploration of the data.
To simplify the process, an additional `tunemix` function enables targeted feature detection by *m/z*, automatically returning the necessary drift times.
For this usage, the data corresponding to a reference tune mix file must be supplied.
The additional parameters `mz_tol` and `dt_tol` are used to refine the selection by *m/z* (in parts per million) and drift time (as a relative fraction), respectively.

In [4]:
# Load tune mix data
tune_pos = deimos.load('example_tune_pos.h5', key='ms1')

# Perform calibration
ccs_cal_pos = deimos.calibration.tunemix(tune_pos,
                                         mz=[118.086255, 322.048121, 622.028960, 922.009798, 1221.990636, 1521.971475],
                                         ccs=[121.3, 153.7, 203, 243.6, 282.2, 317],
                                         q=[1, 1, 1, 1, 1, 1],
                                         buffer_mass=28.013, mz_tol=200E-6, dt_tol=0.04)

# R-squared
print('r-squared:\t', ccs_cal_pos.fit['r'] ** 2)

r-squared:	 0.9999784552958121


In [5]:
# Load tune mix data
tune_neg = deimos.load('example_tune_neg.h5', key='ms1')

# Perform calibration
ccs_cal_neg = deimos.calibration.tunemix(tune_neg,
                                         mz=[301.998139, 601.978977, 1033.988109, 1333.968947, 1633.949786],
                                         ccs=[140, 180.8, 255.3, 284.8, 319],
                                         q=[1, 1, 1, 1, 1],
                                         buffer_mass=28.013, mz_tol=200E-6, dt_tol=0.04)

# R-squared
print('r-squared:\t', ccs_cal_pos.fit['r'] ** 2)

r-squared:	 0.9999784552958121


Finally, fit parameters `beta` and `tfix` may be supplied directly, for example when using vendor software to determine calibration.

In [6]:
# Calibrate positive mode
ccs_cal_pos = deimos.calibration.calibrate_ccs(beta=0.12991516042484708,
                                               tfix=-0.03528247661068562)

# Calibrate negative mode
ccs_cal_neg = deimos.calibration.calibrate_ccs(beta=0.12987753172172878,
                                               tfix=0.010482041068836878)

## Travelling Wave IMS

For travelling wave IMS, the relationship between measurement and CCS must be linearized by the natural logarithm, then fit by linear regression.
Usage is similar to the linear case, save specification of the `power` flag, which must be set to `True`. 
The same use cases -- supplying calibrant coordinates directly, using a tune mix data file, or supply beta and tfix directly, all still apply. 
The below was for positive mode data, values supplied directly.

In [7]:
# Load tune data
twave_tune = deimos.load('example_tune_twave.h5', key='ms1')
twave_tune

,mz,ta,ccs,charge
0,556.271,6.93710,229.787425,1
1,472.314,6.83092,228.687678,1
2,455.284,5.79406,208.789013,1
3,380.212,4.89843,191.690478,1
4,311.075,3.76733,168.391880,1
5,215.054,2.75120,146.793283,1
6,195.082,2.45407,138.193775,1
7,152.066,2.12313,130.394467,1


In [8]:
# Perform calibration
ccs_cal_twave = deimos.calibration.calibrate_ccs(mz=twave_tune['mz'],
                                                 ta=twave_tune['ta'],
                                                 ccs=twave_tune['ccs'],
                                                 q=twave_tune['charge'],
                                                 buffer_mass=28.013,
                                                 power=True)

# R-squared
print('r-squared:\t', ccs_cal_twave.fit['r'] ** 2)

r-squared:	 0.9997453731735061


## Apply

The calibration is applied in the same way across IMS types.
In each above case, a instance of a `CCSCalibration` object is returned, ensuring the appropriate internal model (linear versus power) is consisently applied.
To produce CCS values, simply supply *m/z*, drift time, and charge to the `arrival2ccs` method.

In [9]:
ccs_cal_pos.arrival2ccs(mz=322.048121, ta=19.13204, q=1)

153.8201287278313

The function also accepts vectors of values such that entire data frames may be processed.

In [10]:
ccs_cal_pos.arrival2ccs(mz=tune_pos['mz'], ta=tune_pos['drift_time'], q=1)

array([3.25792330e-01, 3.25790167e-01, 3.25788007e-01, ...,
       5.44737609e+02, 5.43764661e+02, 5.43764615e+02])

Similarly, the `ccs2arrival` method may be used to compute drift time from *m/z* and CCS.

In [11]:
ccs_cal_pos.ccs2arrival(mz=322.048121, ccs=153.82, q=1)

19.132023959395465